In [ ]:
from notebook_prelude import *

In [ ]:
#experiment_helper.save_experiment_params_as_experiment_config()
#experiment_helper.get_experiment_config(experiment_helper.EXPERIMENT_CONFIG_ALL)
#experiment_helper.get_experiment_config('configs/param_grid_configs/graph.example.yaml')
experiment_helper.get_all_task_typ_params_flat()